# Experiment on JupyterHub Template


**In this notebook:**

* Load original mri data + aneurysm mask
* Resample Images to 1.5 mm Voxelsize
* Filter images based on size
* Train uunet network to predict aneurysm mask
* Evaluate mask

**Todo:**


##  Dependencies
Install, load, and initialize all required dependencies for this experiment.

### Install Dependencies

In [1]:
import sys

!{sys.executable} -m pip install -q -e ../utils/

## System libraries

In [2]:
from __future__ import absolute_import, division, print_function
import logging, os, sys

# Enable logging
logging.basicConfig(format='[%(levelname)s] %(message)s', level=logging.INFO, stream=sys.stdout)

# Re-import packages if they change
%load_ext autoreload
%autoreload 2

# Recursion Depth
import sys
sys.setrecursionlimit(10000)

# Intialize tqdm to always use the notebook progress bar
import tqdm
tqdm.tqdm = tqdm.tqdm_notebook

# Third-party libraries
import comet_ml
import numpy as np
import pandas as pd
import nilearn.plotting as nip
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import collections
%matplotlib inline
plt.rcParams["figure.figsize"] = (12,6)
%config InlineBackend.figure_format='retina'  # adapt plots for retina displays
import git


# Project utils
import aneurysm_utils
from aneurysm_utils import evaluation, training

/group/cake/.venv/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [3]:
env = aneurysm_utils.Environment(project="ML_in_MIP", root_folder="/group/cake")
env.cached_data["comet_key"] = "EGrR4luSis87yhHbs2rEaqAWs" 
env.print_info()

Environment Info:

Library Version: 0.1.0
Configured Project: ML_in_MIP

Folder Structure: 
- Root folder: /group/cake
 - Project folder: /group/cake/ML_in_MIP
 - Datasets folder: /group/cake/ML_in_MIP/datasets
 - Models folder: /group/cake/ML_in_MIP/models
 - Experiments folder: /group/cake/ML_in_MIP/experiments


[autoreload of aneurysm_utils.environment failed: Traceback (most recent call last):
  File "/group/cake/.venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/group/cake/.venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/tu-ramonabendias/.pyenv/versions/3.8.10/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/tu-ramonabendias/.pyenv/versions/3.8.10/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 844, in exec_module
  File "<frozen importlib._bootstrap_external>", line 981, in get_code
  File "<frozen importlib._bootstrap_external>", line 911, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_